In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-02-08 10:34:41.598957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 10:34:41.845742: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-08 10:34:44.598796: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '2006-01-01','2019-12-31')
kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1

kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)
print(kospi_train.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2006-01-03  1393.140015  1394.920044  1376.170044  1394.869995  1394.869995   
2006-01-04  1408.189941  1412.150024  1400.079956  1402.109985  1402.109985   
2006-01-05  1405.310059  1405.310059  1378.959961  1395.510010  1395.510010   
2006-01-06  1399.229980  1412.790039  1395.260010  1412.780029  1412.780029   
2006-01-09  1421.900024  1421.900024  1405.050049  1408.329956  1408.329956   

              Volume   Change1  
Date                            
2006-01-03  541300.0  0.004031  
2006-01-04  528000.0  0.005190  
2006-01-05  538100.0 -0.004707  
2006-01-06  587900.0  0.012375  
2006-01-09  531500.0 -0.003150  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(3442, 7)


In [4]:
kospi_test = fdr.DataReader('KS11', '2020-01-01','2022-10-31')
kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1

kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)
print(kospi_test.shape)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2020-01-03  2192.580078  2203.379883  2165.389893  2176.459961  2176.459961   
2020-01-06  2154.969971  2164.419922  2149.949951  2155.070068  2155.070068   
2020-01-07  2166.600098  2181.620117  2164.270020  2175.540039  2175.540039   
2020-01-08  2156.270020  2162.320068  2137.719971  2151.310059  2151.310059   
2020-01-09  2182.199951  2186.449951  2172.159912  2186.449951  2186.449951   

            Volume   Change1  
Date                          
2020-01-03  631600  0.000593  
2020-01-06  592700 -0.009828  
2020-01-07  568200  0.009499  
2020-01-08  913800 -0.011137  
2020-01-09  592600  0.016334  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Change1'], dtype='object')
(696, 7)


In [5]:
kospi_train.index = kospi_train.index.strftime('%Y-%m-%d')
print(kospi_train.index)

kospi_test.index = kospi_test.index.strftime('%Y-%m-%d')
print(kospi_test.index)

Index(['2006-01-03', '2006-01-04', '2006-01-05', '2006-01-06', '2006-01-09',
       '2006-01-10', '2006-01-11', '2006-01-12', '2006-01-13', '2006-01-16',
       ...
       '2019-12-16', '2019-12-17', '2019-12-18', '2019-12-19', '2019-12-20',
       '2019-12-23', '2019-12-24', '2019-12-26', '2019-12-27', '2019-12-30'],
      dtype='object', name='Date', length=3442)
Index(['2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09',
       '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16',
       ...
       '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21', '2022-10-24',
       '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-31'],
      dtype='object', name='Date', length=696)


In [6]:
import json
embedding = open('/home/work/nlp/embedding/result/news_summary_64_2006.json', encoding='utf-8')
em_dict = json.load(embedding)

In [7]:
ke_vec_train = []
ke_vec_test = []

for index, row in kospi_train.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Close'])
    ke_vec_train.append(a)

for index, row in kospi_test.iterrows():
    a = em_dict[index].copy()
    a.insert(0, row['Close'])
    ke_vec_test.append(a)

In [8]:
print(ke_vec_train[0], ke_vec_test[0])

[1394.869995, 0.08540033549070358, -0.4674851894378662, -0.5362037420272827, -0.25901123881340027, -0.26654699444770813, 0.017446843907237053, 0.3900393843650818, 0.04290249943733215, 0.042930614203214645, -0.05205506831407547, 0.2535240948200226, -0.35728463530540466, -0.3169534206390381, -0.3042212724685669, -0.1683989018201828, 0.057889893651008606, 0.2807113230228424, -0.3238532245159149, 0.167076975107193, 0.08664888888597488, -0.49496519565582275, 0.07980471849441528, 0.1096445620059967, -0.028023697435855865, -0.09358430653810501, -0.30323222279548645, 0.5093612670898438, -0.19800728559494019, -0.3265606760978699, 0.1687026023864746, -0.31198549270629883, -0.46485716104507446, 0.34240132570266724, 0.022288605570793152, 0.16348792612552643, 0.09795103222131729, -0.29747506976127625, 0.10231968015432358, -0.30191770195961, -0.0827576071023941, -0.1480548083782196, -0.3621669411659241, -0.16694578528404236, 0.16708868741989136, 0.22266316413879395, -0.16095221042633057, 0.295243471

In [9]:
kospi_train_c1 = kospi_train['Change1']
kospi_train_c1 = 100 * kospi_train_c1.values

kospi_test_c1 = kospi_test['Change1']
kospi_test_c1 = 100 * kospi_test_c1.values

In [10]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_test_1, y_test_1 = [], []

In [11]:
for i in range(lookback_window, len(ke_vec_train)-1):
    x_train_1.append(ke_vec_train[i - lookback_window:i])
    y_train_1.append(kospi_train_c1[i])
    
for index in range(len(y_train_1)):
    if y_train_1[index] >= 0:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease' 

In [12]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
print(x_train_1.shape, y_train_1.shape)

(3381, 60, 65) (3381,)


In [13]:
for i in range(lookback_window, len(ke_vec_test)-1):
    x_test_1.append(ke_vec_test[i - lookback_window:i])
    y_test_1.append(kospi_test_c1[i])
    
for index in range(len(y_test_1)):
    if y_test_1[index] >= 0:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'    

In [14]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
print(x_test_1.shape, y_test_1.shape)

(635, 60, 65) (635,)


In [15]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_test_1_e = to_categorical(encoder.transform(y_test_1))

print(y_train_1_e.shape, y_test_1_e.shape)

(3381, 2) (635, 2)


In [16]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 65),
        nb_filters=100,
        kernel_size=2,
        dilations=[1, 2, 4, 8, 16],
        use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_2 = Sequential([
    TCN(input_shape=(lookback_window, 65),
        nb_filters=100,
        kernel_size=3,
        dilations=[1, 2, 4, 8, 16],
        use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_2.summary()
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
###################################################################
model_3 = Sequential([
    TCN(input_shape=(lookback_window, 65),
        nb_filters=100,
        kernel_size=4,
        dilations=[1, 2, 4, 8, 16],
        use_batch_norm=True
        ),
    Dense(2, activation='softmax')
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-02-08 10:34:50.769257: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 10:34:51.357502: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22597 MB memory:  -> device: 0, name: CUDA GPU, pci bus id: 0000:d5:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 100)               204600    
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 204,802
Trainable params: 202,802
Non-trainable params: 2,000
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 100)               301100    
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 301,302
Tr

In [17]:
print('Train model_1...')
model_1.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

print('Train model_2...')
model_2.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

print('Train model_3...')
model_3.fit(x_train_1, y_train_1_e, epochs=50, batch_size=32, verbose=1)

Train model_1...
Epoch 1/50


2023-02-08 10:34:58.002037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400
2023-02-08 10:35:03.975888: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-08 10:35:04.071269: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f28e8983220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-08 10:35:04.071364: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): CUDA GPU, Compute Capability 8.6
2023-02-08 10:35:04.086114: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-08 10:35:04.237926: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


106/106 [==============================] - 52s 304ms/step - loss: 0.7753 - accuracy: 0.5152
Epoch 2/50
106/106 [==============================] - 4s 40ms/step - loss: 0.7457 - accuracy: 0.5019
Epoch 3/50
106/106 [==============================] - 4s 36ms/step - loss: 0.7290 - accuracy: 0.5114
Epoch 4/50
106/106 [==============================] - 4s 35ms/step - loss: 0.7214 - accuracy: 0.5102
Epoch 5/50
106/106 [==============================] - 4s 35ms/step - loss: 0.7249 - accuracy: 0.5167
Epoch 6/50
106/106 [==============================] - 4s 34ms/step - loss: 0.7378 - accuracy: 0.5182
Epoch 7/50
106/106 [==============================] - 4s 36ms/step - loss: 0.7029 - accuracy: 0.5383
Epoch 8/50
106/106 [==============================] - 4s 36ms/step - loss: 0.7169 - accuracy: 0.5135
Epoch 9/50
106/106 [==============================] - 4s 37ms/step - loss: 0.7124 - accuracy: 0.5265
Epoch 10/50
106/106 [==============================] - 4s 35ms/step - loss: 0.7033 - accuracy: 0.525

106/106 [==============================] - 4s 35ms/step - loss: 0.6777 - accuracy: 0.5705
Epoch 32/50
106/106 [==============================] - 4s 36ms/step - loss: 0.6733 - accuracy: 0.5856
Epoch 33/50
106/106 [==============================] - 4s 37ms/step - loss: 0.6697 - accuracy: 0.5883
Epoch 34/50
106/106 [==============================] - 4s 37ms/step - loss: 0.6768 - accuracy: 0.5827
Epoch 35/50
106/106 [==============================] - 4s 37ms/step - loss: 0.6679 - accuracy: 0.5856
Epoch 36/50
106/106 [==============================] - 4s 36ms/step - loss: 0.6696 - accuracy: 0.5844
Epoch 37/50
106/106 [==============================] - 4s 36ms/step - loss: 0.6676 - accuracy: 0.5901
Epoch 38/50
106/106 [==============================] - 4s 37ms/step - loss: 0.6661 - accuracy: 0.5945
Epoch 39/50
106/106 [==============================] - 4s 37ms/step - loss: 0.6638 - accuracy: 0.5992
Epoch 40/50
106/106 [==============================] - 4s 35ms/step - loss: 0.6665 - accuracy:

In [ ]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_2.evaluate(x_test_1, y_test_1_e)[1]
model_3.evaluate(x_test_1, y_test_1_e)[1]